In [ ]:
from qiskit.quantum_info import random_unitary
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit.library import RZGate, RYGate, XGate
from qiskit.quantum_info import Operator

import numpy as np

Adapted from Nielsen, M. A., & Chuang, I. L. (2010). Quantum computation and quantum information (10th anniversary ed., p. 181). Cambridge University Press.

This code creates a multi-control $U$ gate, using only 1 qubit and cx gates, such that:

$$ C^n U \, |x\rangle_n \, |y\rangle_1 = 
\begin{cases} 
|x\rangle_n \, U|y\rangle_1, & \text{if } x = (1, 1, \dots, 1), \\
|x\rangle_n \, |y\rangle_1, & \text{otherwise}.
\end{cases}$$

#############################################################################

(1)

To construct the singly controlled unitary gate, recall that any unitary can be written using the ZYZ decomposition: 
$$U = e^{i \phi} R_z(\beta) R_y(\gamma) R_z(\delta) = e^{i \alpha}U' = \begin{bmatrix} e^{i(\alpha - \beta / 2 - \delta / 2)}\cos(\gamma /2) & - e^{i(\alpha - \beta / 2 + \delta / 2)}\sin(\gamma / 2) \\ e^{i(\alpha + \beta / 2 - \delta / 2)}\sin(\gamma /2) & e^{i(\alpha + \beta / 2 + \delta / 2)}\cos(\gamma /2) \end{bmatrix}$$

$$\implies \alpha = \det(U)/2$$

$$\implies \gamma = 2 \arccos(|a|) \quad \text{where} \quad a = U_{11}$$

Letting $\theta_1 = \beta / 2 - \delta / 2$ and
$\theta_2 = -\beta / 2 + \delta / 2$, we get

$$\implies \beta = -(\theta_1 + \theta_2)$$

$$\implies \delta = \theta_2 - \theta_1$$

In [ ]:
def cv_gate(V, qc, control, target):
    """
    Adds a singly controlled unitary gate to a quantum circuit
    
    Uses 1 qubit gates to construct the ZYZ decomposition of a unitary, V, and contructs a controlled unitary gate

    Inputs:
    V =  any unitary
    qc = quantum circuit
    control = control qubit
    target = target qubits

    This circuit is constructed following Nielsen and Chuang figure 4.6

    Output: singly controlled unitary gate
    """
    detU = np.linalg.det(U)

    # alpha
    alpha = np.angle(detU) / 2

    # U_prime contains beta, delta, and gamma
    U_prime = U * np.exp(-1j * alpha)

    # select the first (1,1) and (1,2) entry to U'
    a, b = U_prime[0, 0], U_prime[0, 1]

    gamma = 2 * np.arccos(np.abs(a))

    # Fix angles carefully
    theta1 = np.angle(a)  # - (beta + delta)/2
    theta2 = np.angle(b)  # - (beta - delta)/2

    beta = -(theta1 + theta2)
    delta = theta2 - theta1

    # build_ABC_circuit(alpha, beta, gamma, delta)
    A = RZGate(beta)
    B = RYGate(gamma)
    C = RZGate(delta)
    X = XGate()

    # convert to matrix form
    A_mat = Operator(A).data
    B_mat = Operator(B).data
    C_mat = Operator(C).data
    X_mat = Operator(X).data
    
    # insert X between A and B, and between B and C
    U_reconstructed = A_mat @ X_mat @ B_mat @ X_mat @ C_mat
    U_full = np.exp(1j * alpha) * U_reconstructed
    
    # check that U in a 2x2 numpy array
    if not np.allclose(U_full, U, atol=1e-8):
        raise ValueError("ABC decomposition failed")

    qc.rz(beta, target)
    qc.cx(control, target)
    qc.ry(gamma, target)
    qc.cx(control, target)
    qc.rz(delta, target)

    return qc

### Example usage and circuit drawing

# random unitary
#U = random_unitary(2).data

# quantum circuit with 3 qubits
#qc2 = QuantumCircuit(3)  # qubit 0 = control, qubit 1 = target

# control on the first qubit
#control = 0

# target the third qubit
#target = 2

# draw the circuit
#qc = cv_gate(U, qc2, control, target)
#qc.draw(output="mpl", style="bw")

In [ ]:
def toffoli(qc, control1, control2, target):
    """
    Adds a Toffoli gate to a quantum circuit using only H, T, T†, and CNOTs.

    Inputs: 
    qc = quantum circuit
    control1 = first control qubit
    control2 = second control qubit
    target = target qubits

    This circuit is constructed following Nielsen and Chuang figure 4.9

    Output: Toffoli gate 
    """
    # (1)
    qc.h(target)    
    qc.cx(control2, target)
    qc.tdg(target)
    qc.cx(control1, target)
    qc.t(target)
    qc.cx(control2, target)
    qc.tdg(target)
    qc.cx(control1, target)
    qc.t(target)
    qc.h(target)
    qc.cx(control1, control2)
    qc.tdg(control2)
    qc.cx(control1, control2)
    qc.t(control1)
    qc.t(control2)

    #print(np.real(np.round(np.array(Operator(qc)), 3)).astype(int)) # converting to real integer type just for better readablility

    return qc

### Example usage and circuit drawing

# quantum circuit with 3 qubits
#qc = QuantumCircuit(3)

# draw the circuit
#toffoli(qc, 0, 1, 2) # control the first 2 qubits and target the third
#qc.draw('mpl', style='bw')

In [ ]:
def MCU(n, U):
    """
    Creates a multi-controlled unitary gate quantum circuit

    Inputs: 
    n = number of qubits
    U = arbitary unitary

    (1) If theres only 1 control and a target qubit, we perform the ZYZ decomposition
    (2) Construct the circuit in figure 4.10 of Nielsen and Chuang using toffolis to target the ancillas
    (3) Apply the singly controlled unitary gate, that we constructed above, to the target qubit, controlled by the last ancilla
    (4) Reverse the ancilla
    

    Output: Multi-controlled unitary quantum circuit
    """
    
    # (1)
    if n == 1:
        controls = QuantumRegister(n, 'q')
        target = QuantumRegister(1, 't')
        qc = QuantumCircuit(controls, target)
        cv_gate(U, qc, controls[0], target)
    else:
    # (2)
        # Create a quantum control register with n qubits
        controls = QuantumRegister(n, 'q')
    
        # Create ancilla register
        ancilla = AncillaRegister(n - 1, name="a")
    
        # Create a target register with 1 qubit
        target = QuantumRegister(1, 't')
    
        # Create the quantum circuit
        qc = QuantumCircuit(controls, ancilla, target)

        # control the first ancilla with the first two qubits. Then use toffolis using the next control and the previous ancilla. 
        toffoli(qc, 0, 1, ancilla[0])
        for i in range(n-2):
            toffoli(qc, i+2, ancilla[i], ancilla[i+1])

        # (3)
        # Apply the unitary to the target qubit, controlled by the last ancilla
        cv_gate(U, qc, ancilla[n-2], target)

        # (4)
        # Reverse ancilla
        for i in range(n-2):
            toffoli(qc, controls[n-i - 1], ancilla[n - 3 - i], ancilla[n - 2 - i])
        toffoli(qc, 0, 1, ancilla[0])
        
    return qc

### Example use and circuit drawing

# number of qubits 
#n = 3

# random unitary
#U = random_unitary(2).data

# draw circuit
#qc = MCU(n,U)
#qc.draw('mpl')

In [ ]:
# benchmarking
for n in range(2, 6):
    U = random_unitary(2).data
    qc = MCU(n, U)  # Already using only basic gates

    gate_count = qc.count_ops()
    depth = qc.depth()
    ancilla_count = qc.num_ancillas

    print(f"n = {n}")
    print(f"Gate count: {gate_count} (Grows as: O(n^2))")
    print(f"Circuit depth: {depth} (Grows as: O(n))")
    print(f"Number of ancillas: {ancilla_count} (Grows as: O(n))")
    print()